In [2]:
import pandas as pd
import numpy as np

In [3]:
#reading csv file into pandas dataframe and naming df
df = pd.read_csv("/Users/beans/Desktop/Mod-4-Project/data/hour.csv")

#understanding values within df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [ ]:
#making season column more legibile by associating numbers with season
#shown by readme when downloaded dataset
df['season'] = df['season'].replace(1,'winter')
df['season'] = df['season'].replace(2,'spring')
df['season'] = df['season'].replace(3,'summer')
df['season'] = df['season'].replace(4,'fall')



,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
6250,6251,2011-09-23,fall,0,9,0,0,5,1,2,0.62,0.5455,0.94,0.0896,11,41,52
6251,6252,2011-09-23,fall,0,9,1,0,5,1,2,0.60,0.5000,1.00,0.0000,2,29,31
6252,6253,2011-09-23,fall,0,9,2,0,5,1,2,0.60,0.5000,1.00,0.1045,6,14,20
6253,6254,2011-09-23,fall,0,9,3,0,5,1,2,0.60,0.5000,1.00,0.0000,3,5,8
6254,6255,2011-09-23,fall,0,9,4,0,5,1,3,0.60,0.5000,1.00,0.0000,6,7,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17112,17113,2012-12-20,fall,1,12,19,0,4,1,3,0.34,0.3182,0.71,0.2537,3,115,118
17113,17114,2012-12-20,fall,1,12,20,0,4,1,3,0.34,0.3333,0.76,0.1642,1,49,50
17114,17115,2012-12-20,fall,1,12,21,0,4,1,3,0.34,0.3333,0.76,0.1642,1,25,26
17115,17116,2012-12-20,fall,1,12,22,0,4,1,3,0.34,0.3333,0.87,0.1940,1,20,21
